In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import ISLP
from ISLP import load_data
from ISLP import models

In [3]:
carseats=load_data('Carseats')
carseats.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [9]:
from ISLP.models import ModelSpec as MS
X = carseats[['CompPrice','Income','Advertising','Population','Age','Education']]
y = carseats['Sales']
X = sm.add_constant(X)
M = sm.OLS(y,X).fit()
print(M.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     11.51
Date:                Fri, 06 Dec 2024   Prob (F-statistic):           7.03e-12
Time:                        23:45:56   Log-Likelihood:                -949.96
No. Observations:                 400   AIC:                             1914.
Df Residuals:                     393   BIC:                             1942.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           7.2737      1.520      4.785      

In [23]:
def get_p_values(data, response_var, predictor_vars):
    p_values = {
        var: sm.OLS(data[response_var], sm.add_constant(data[[var]])).fit().pvalues[var]
        for var in predictor_vars
    }
    p_values_df = pd.DataFrame(p_values.items(), columns=['Variable', 'p-value']).sort_values('p-value')
    return p_values_df

predictor_vars = ['CompPrice', 'Income', 'Advertising', 'Population', 'Age', 'Education']
response_var = 'Sales'
p_values_df = get_p_values(carseats, response_var, predictor_vars)
print(p_values_df)

      Variable       p-value
2  Advertising  4.377677e-08
4          Age  2.788950e-06
1       Income  2.309670e-03
0    CompPrice  2.009398e-01
5    Education  2.999442e-01
3   Population  3.139816e-01


In [25]:
#2
#control for type 1 error
#H0: beta1 = 0 (The predictor X has no effect on Sales)
alpha = 0.05

rejected_type1 = p_values_df[p_values_df['p-value'] < alpha]

print("\nHypotheses Rejected under Type I Error Control (α = 0.05):")
print(rejected_type1)


Hypotheses Rejected under Type I Error Control (α = 0.05):
      Variable       p-value
2  Advertising  4.377677e-08
4          Age  2.788950e-06
1       Income  2.309670e-03


In [29]:
#3
from statsmodels.stats.multitest import multipletests
m = len(p_values_df)
# Apply Bonferroni correction
reject_bonferroni, pvals_corrected_bonf, _, _ = multipletests(p_values_df['p-value'], alpha, method='bonferroni')
p_values_df['Bonferroni'] = reject_bonferroni

rejected_fwer = p_values_df[p_values_df['Bonferroni']]

print("\nHypotheses Rejected under FWER Control")
print(rejected_fwer[['Variable', 'p-value']])


Hypotheses Rejected under FWER Control
      Variable       p-value
2  Advertising  4.377677e-08
4          Age  2.788950e-06
1       Income  2.309670e-03


In [31]:
#4.
# Apply Benjamini-Hochberg correction
reject_bh, pvals_corrected_bh, _, _ = multipletests(p_values_df['p-value'], alpha=0.2, method='fdr_bh')
p_values_df['BH FDR'] = reject_bh

rejected_fdr = p_values_df[p_values_df['BH FDR']]

print("\nHypotheses Rejected under FDR Control :")
print(rejected_fdr[['Variable', 'p-value']])


Hypotheses Rejected under FDR Control :
      Variable       p-value
2  Advertising  4.377677e-08
4          Age  2.788950e-06
1       Income  2.309670e-03


#in all three adverstising, age, income are significant